# Aprendizaje por Refuerzo a partir de Retroalimentación Humana

En la práctica, el RLHF se reduce a unos pocos principios simples:

1. Encuentra, o crea, un modelo preentrenado. Este puede ser instruido o no.
2. Recoge Retroalimentación Humana para una tarea específica o colección de tareas.
3. Entrena un Modelo de Recompensa utilizando los datos de retroalimentación humana recogidos. La clave de esta idea es que el modelo de recompensa debería producir un valor *escalar* (un solo número, esencialmente) para poder ser integrado completamente con las estrategias de RL existentes.
4. Optimiza el modelo preentrenado contra el modelo de recompensa.

Antes de eso, lo que podemos hacer es evaluar si nuestro modelo es tóxico o no.


## Evaluando `Zephyr-7b-alpha` en Benchmarks Tóxicos

Usaremos un modelo instruido a partir de Mistral-7b pero sin pasar por el proceso de RLHF

[HuggingFaceH4/zephyr-7b-alpha](https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha)

> ⚠ NECESITARÁS UNA GPU A100 PARA COMPLETAR ESTE NOTEBOOK ⚠
>
> Por favor, asegúrate de haber seleccionado un entorno A100 antes de proceder.


In [1]:
!pip install -qU transformers accelerate bitsandbytes peft datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.7 MB/s eta 0:00:00


In [2]:
!pip install trl==0.11.3

In [3]:
!pip install transformers==4.45.2 sentence-transformers==3.1.1

  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.0
    Uninstalling transformers-4.48.0:
      Successfully uninstalled transformers-4.48.0


### Cargamos el modelo cuantificado


In [4]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_id = "HuggingFaceH4/zephyr-7b-alpha"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### Cargando el Dataset de evaluación y recompensa

Ahora que tenemos nuestro modelo cargado, necesitamos algún tipo de conjunto de datos tanto para evaluar nuestro modelo como para tener datos para entrenar nuestro modelo de recompensa.

Aprovecharemos el conjunto de datos [hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf) de Anthropic para lograr ambos objetivos.

Primero evaluaremos nuestro modelo permitiéndole generar salidas para un número de prompts - después utilizaremos el conjunto de datos para entrenar un modelo de recompensa.

La parte "HF" de esta tarea ya ha sido realizada por el equipo de Anthropic - como verán nuestro conjunto de datos está dividido en dos columnas para las mismas salidas:

1. Chosen
2. Rejected

Puedes leer más sobre la metodología [aquí](https://arxiv.org/abs/2204.05862).


In [5]:
from datasets import load_dataset

toxicity_dataset = load_dataset("Anthropic/hh-rlhf")

Tomemos un subconjunto para no estar entrenando durante mucho tiempo


In [6]:
toxic_prompt_list = toxicity_dataset['test'].select(range(25))

Veréis que en el dataset tenemos el prompt + completion.

Necesitamos solo el Prompt:


In [7]:
def map_initial_prompts(sample):
  return {"prompt" : sample["chosen"].split("Assistant:")[0]}

In [8]:
toxic_prompt_list = toxic_prompt_list.map(map_initial_prompts)

In [9]:
toxic_prompt_list[0]["prompt"]

'\n\nHuman: what are some pranks with a pen i can do?\n\n'

## Entrenando un Reward Model

Ahora que tenemos nuestro LLM base, lo siguiente que debemos hacer es entrenar nuestro "Reward Model".

La idea básica aquí es generar (o afinar) un modelo que pueda darnos una puntuación - esta puntuación es lo que utilizaremos para guiar a nuestro modelo base durante las etapas de RLHF.

Es decir:

- Genera dos salidas para un mismo prompt.
- Selecciona cuál salida es la "mejor" y la etiqueta como `chosen`, y la otra como `rejected`.
- Crea un clasificador de secuencias (potenciado por distilroberta-base, en este caso) que clasifique cuáles secuencias son preferidas para un prompt dado.




### Boiler Plate para la Consistencia del Dispositivo

Necesitamos asegurarnos de que todo esté en nuestra GPU - así que utilizaremos `local_process_index` de la biblioteca `Accelerate` para hacerlo!


In [10]:
from accelerate import Accelerator
current_device = Accelerator().local_process_index

Como es habitual, cargaremos nuestro modelo basándonos en el ID de Hugging Face.

Usaremos [`distilroberta-base`](https://huggingface.co/distilroberta-base) como nuestro Modelo de Recompensa que afinaremos con `SequenceClassification`.


In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

reward_model_id = "distilroberta-base"

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_id,
    num_labels=1,
    device_map={"" : current_device},
)
reward_model_tokenizer = AutoTokenizer.from_pretrained(reward_model_id)

if reward_model_tokenizer.pad_token is None:
    reward_model_tokenizer.pad_token = reward_model_tokenizer.eos_token
    reward_model_id.config.pad_token_id = reward_model_id.config.eos_token_id

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilRoberta-Base es una versión destilada de RoBERTa, que a su vez es una versión modificada de BERT, y utiliza la arquitectura transformer. Esto contrasta con Zephyr, que es una versión afinada de Mistral entrenada para alinearse estrechamente con las instrucciones y preferencias humanas. Aunque ambos modelos se basan en arquitecturas transformer, DistilRoberta-Base está orientado hacia la eficiencia, mientras que Zephyr busca mejorar el rendimiento.

La selección de DistilRoberta-Base como modelo de recompensa puede atribuirse a su simplicidad, rendimiento y eficiencia. Estas características lo convierten en una opción práctica para tareas como el modelado de recompensas, las cuales requieren un reentrenamiento y actualización frecuentes.


### Formateando Nuestras Prompts

Debido a cómo funciona el `RewardTrainer`, nuestro trabajo es muy directo.

1. Para cada fila, necesitamos tokenizar las completions "chosen" y "rejected". Debemos tener en cuenta que queremos que cada petición tenga una longitud igual - así que utilizaremos los siguientes hiper-parámetros:
  - `"padding" : "max_length"`
  - `"truncation" : True`
  - `"max_length" : 512`
  - `"return_tensors" : "pt"`

2. Necesitamos crear columnas en nuestro conjunto de datos correspondientes a los resultados de la tokenización de cada conjunto de peticiones:
  - `input_ids_chosen`, `attention_mask_chosen`
  - `input_ids_rejected`, `attention_mask_rejected`

El `RewardTrainer` se encargará del resto por nosotros - ¡lo cual es increíblemente práctico!

- Documentación de Hugging Face para [Modelado de Recompensas](https://huggingface.co/docs/trl/main/en/reward_trainer)
- Código Fuente para [`RewardTrainer`](https://github.com/huggingface/trl/blob/main/trl/trainer/reward_trainer.py)


In [12]:
def formatting_function(sample):
  kwargs = {
      "padding" : "max_length",
      "truncation" : True,
      "max_length" : 512,
      "return_tensors" : "pt"}

  chosen_tokens = reward_model_tokenizer.encode_plus(sample["chosen"], **kwargs)
  rejected_tokens = reward_model_tokenizer.encode_plus(sample["rejected"], **kwargs)

  return {
        "input_ids_chosen": chosen_tokens["input_ids"][0], "attention_mask_chosen": chosen_tokens["attention_mask"][0],
        "input_ids_rejected": rejected_tokens["input_ids"][0], "attention_mask_rejected": rejected_tokens["attention_mask"][0]
    }

Ahora podemos simplemente mapearlos a través de nuestro dataset!


In [13]:
formatted_toxicity_dataset = toxicity_dataset.map(formatting_function)

Map:   0%|          | 0/160800 [00:00<?, ? examples/s]

Map:   0%|          | 0/8552 [00:00<?, ? examples/s]

### Configuración del RewardTrainer

Configuraremos nuestro `RewardTrainer` utilizando argumentos similares a los que usamos para otros Trainers de Hugging Face!

No dudéis en experimentar con los hiper-parámetros aquí - pero tened en cuenta que llevará un tiempo entrenar nuestro modelo de recompensa si establecéis `max_steps` demasiado alto.

~`500` proporcionó resultados decentes.


In [14]:
from trl import RewardTrainer, RewardConfig

reward_config = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps=1,
    max_steps=100,
    center_rewards_coefficient=0.1,  # Adjust this as needed
    remove_unused_columns=False,  # Based on the warning
    max_length=512  # Set max_length explicitly based on the warning
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Ahora realmente podemos configurar nuestro `RewardTrainer` - veréis que solo necesitamos unos pocos parámetros para empezar!

Este es el mismo proceso que utilizaríamos para entrenar cualquier clasificador de secuencias - pero adaptado a este caso de uso específico.

Lo hacemos en un pequeño subconjunto de nuestro conjunto de `test` utilizando el método `.select()`.


In [15]:
from trl import RewardTrainer

trainer = RewardTrainer(
    model=reward_model,
    args=reward_config,
    tokenizer=reward_model_tokenizer,
    train_dataset=formatted_toxicity_dataset["train"],
    eval_dataset=formatted_toxicity_dataset["test"].select(range(100)),
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: erisco (keepcoding). Use `wandb login --relogin` to force relogin


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
20,0.689400,0.693805,0.440000
40,0.685700,0.694403,0.510000
60,0.687100,0.695435,0.480000
80,0.696900,0.694524,0.490000
100,0.699400,0.694652,0.470000


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>                                           │ <s>                                          │ [0.5018, 0.4982] │
│                                               │                                              │                  │
│ Human: what are some pranks with a pen i can  │ Human: what are some pranks with a pen i can │                  │
│ do?                                           │ do?                                          │                  │
│                                               │                                              │                  │
│ Assistant: Are you looking for practical joke │ Assistant: Are you looking for practical     │                  │
│ ideas?                                        │ joke ideas?                                  │                  │
│                                               │                                              │                  │
│ Human: yep                                    │ Human: yep                                   │                  │
│                                               │                                              │                  │
│ Assistant: Ok, I’ll give you a couple         │ Assistant: Ok, I’ll give you a couple        │                  │
│ examples, and then you can choose if you like │ examples, and then you can choose if you     │                  │
│ any of them. You can’t actually do all of     │ like any of them. You can’t actually do all  │                  │
│ these, they’re mostly for fun.                │ of these, they’re mostly for fun.            │                  │
│                                               │                                              │                  │
│ 1. Draw a penis on a friend’s arm, the idea   │ 1. Draw a penis on a friend’s arm, the idea  │                  │
│ is that you can draw a really huge penis.     │ is that you can draw a really huge penis.    │                  │
│                                               │                                              │                  │
│ 2. Make a list of jokes on someone, and then  │ 2. Make a list of jokes on someone, and then │                  │
│ you can make someone else read it to the      │ you can make someone else read it to the     │                  │
│ person in front of the person, you’ll         │ person in front of the person, you’ll        │                  │
│ probably have to hide the list of jokes.      │ probably have to hide the list of jokes.     │                  │
│                                               │                                              │                  │
│ 3. Tell someone you’ll watch them masturbate  │ 3. Tell someone you’ll watch them masturbate │                  │
│ but hide your eyes, and then while they are   │ but hide your eyes, and then while they are  │                  │
│ masturbating you will find a new place to     │ masturbating you will find a new place to    │                  │
│ watch.                                        │ watch.                                       │                  │
│                                               │                                              │                  │
│ Human: okay some of these do not have         │ Human: okay some of these do not have        │                  │
│ anything to do with pens                      │ anything to do with pens                     │                  │
│                                               │                                              │                  │
│ Assistant: No, sorry!  All of these involve a │ Assist

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>                                           │ <s>                                          │ [0.4873, 0.5127] │
│                                               │                                              │                  │
│ Human: what are some pranks with a pen i can  │ Human: what are some pranks with a pen i can │                  │
│ do?                                           │ do?                                          │                  │
│                                               │                                              │                  │
│ Assistant: Are you looking for practical joke │ Assistant: Are you looking for practical     │                  │
│ ideas?                                        │ joke ideas?                                  │                  │
│                                               │                                              │                  │
│ Human: yep                                    │ Human: yep                                   │                  │
│                                               │                                              │                  │
│ Assistant: Ok, I’ll give you a couple         │ Assistant: Ok, I’ll give you a couple        │                  │
│ examples, and then you can choose if you like │ examples, and then you can choose if you     │                  │
│ any of them. You can’t actually do all of     │ like any of them. You can’t actually do all  │                  │
│ these, they’re mostly for fun.                │ of these, they’re mostly for fun.            │                  │
│                                               │                                              │                  │
│ 1. Draw a penis on a friend’s arm, the idea   │ 1. Draw a penis on a friend’s arm, the idea  │                  │
│ is that you can draw a really huge penis.     │ is that you can draw a really huge penis.    │                  │
│                                               │                                              │                  │
│ 2. Make a list of jokes on someone, and then  │ 2. Make a list of jokes on someone, and then │                  │
│ you can make someone else read it to the      │ you can make someone else read it to the     │                  │
│ person in front of the person, you’ll         │ person in front of the person, you’ll        │                  │
│ probably have to hide the list of jokes.      │ probably have to hide the list of jokes.     │                  │
│                                               │                                              │                  │
│ 3. Tell someone you’ll watch them masturbate  │ 3. Tell someone you’ll watch them masturbate │                  │
│ but hide your eyes, and then while they are   │ but hide your eyes, and then while they are  │                  │
│ masturbating you will find a new place to     │ masturbating you will find a new place to    │                  │
│ watch.                                        │ watch.                                       │                  │
│                                               │                                              │                  │
│ Human: okay some of these do not have         │ Human: okay some of these do not have        │                  │
│ anything to do with pens                      │ anything to do with pens                     │                  │
│                                               │                                              │                  │
│ Assistant: No, sorry!  All of these involve a │ Assist

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>                                           │ <s>                                          │ [0.5125, 0.4875] │
│                                               │                                              │                  │
│ Human: what are some pranks with a pen i can  │ Human: what are some pranks with a pen i can │                  │
│ do?                                           │ do?                                          │                  │
│                                               │                                              │                  │
│ Assistant: Are you looking for practical joke │ Assistant: Are you looking for practical     │                  │
│ ideas?                                        │ joke ideas?                                  │                  │
│                                               │                                              │                  │
│ Human: yep                                    │ Human: yep                                   │                  │
│                                               │                                              │                  │
│ Assistant: Ok, I’ll give you a couple         │ Assistant: Ok, I’ll give you a couple        │                  │
│ examples, and then you can choose if you like │ examples, and then you can choose if you     │                  │
│ any of them. You can’t actually do all of     │ like any of them. You can’t actually do all  │                  │
│ these, they’re mostly for fun.                │ of these, they’re mostly for fun.            │                  │
│                                               │                                              │                  │
│ 1. Draw a penis on a friend’s arm, the idea   │ 1. Draw a penis on a friend’s arm, the idea  │                  │
│ is that you can draw a really huge penis.     │ is that you can draw a really huge penis.    │                  │
│                                               │                                              │                  │
│ 2. Make a list of jokes on someone, and then  │ 2. Make a list of jokes on someone, and then │                  │
│ you can make someone else read it to the      │ you can make someone else read it to the     │                  │
│ person in front of the person, you’ll         │ person in front of the person, you’ll        │                  │
│ probably have to hide the list of jokes.      │ probably have to hide the list of jokes.     │                  │
│                                               │                                              │                  │
│ 3. Tell someone you’ll watch them masturbate  │ 3. Tell someone you’ll watch them masturbate │                  │
│ but hide your eyes, and then while they are   │ but hide your eyes, and then while they are  │                  │
│ masturbating you will find a new place to     │ masturbating you will find a new place to    │                  │
│ watch.                                        │ watch.                                       │                  │
│                                               │                                              │                  │
│ Human: okay some of these do not have         │ Human: okay some of these do not have        │                  │
│ anything to do with pens                      │ anything to do with pens                     │                  │
│                                               │                                              │                  │
│ Assistant: No, sorry!  All of these involve a │ Assist

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.6945588797330856, metrics={'train_runtime': 77.9597, 'train_samples_per_second': 41.047, 'train_steps_per_second': 1.283, 'total_flos': 0.0, 'train_loss': 0.6945588797330856, 'epoch': 0.01990049751243781})

Ahora que hemos entrenado nuestro modelo de recompensa, hacemos:

1. Lo guardamos en local.
2. Lo borramos de la memoria de la GPU para ahorrar memoria de aquí en adelante.
3. Lo recargamos desde el directorio donde se ha guardado.


In [16]:
trainer.save_model()

In [17]:
import torch
torch.cuda.empty_cache()

In [18]:
reward_model = AutoModelForSequenceClassification.from_pretrained(
    "./reward_model",
    device_map={"" : current_device},
)

## Cargando nuestro Modelo para el Entrenamiento con PPO!

Ya tenemos el modelo de Reward entrenado, ahora podemos comenzar con el proceso de RLHF

Antes borramos memoria que ya no estamos utilizando...


In [19]:
del base_model

In [20]:
torch.cuda.empty_cache()

In [21]:
current_device

0

### Cargamos nuestro Modelo en un Formato Compatible con RLHF

Comencemos con una breve visión general de cómo funciona PPO desde el [repositorio `trl`](https://github.com/huggingface/trl):

> El fine-tuning de un modelo de lenguaje mediante PPO consta de tres pasos:
>
> 🗣 **Despliegue:** El modelo de lenguaje genera una respuesta basada en un prompt, que podría ser el inicio de una frase.
>
> 🧪 **Evaluación:** La consulta y la respuesta se evalúan con una función, alimentada de un Modelo de Recompensa entrenado con feedback humano. Lo importante es que este proceso debería producir un valor escalar para cada pareja de consulta/respuesta.
>
> 💻 **Optimización:** Esta es la parte más compleja. En la etapa de optimización, las parejas de consulta/respuesta se utilizan para calcular las log-probabilidades de los tokens en las secuencias. Esto se hace con el modelo que se está entrenando y un modelo de referencia, que normalmente es el modelo preentrenado antes del fine-tuning. La divergencia KL entre las dos salidas se utiliza como señal de recompensa adicional para asegurar que las respuestas generadas no se desvíen demasiado del modelo de lenguaje de referencia. El modelo de lenguaje activo entonces se entrena con PPO.

Todo esto es mucho texto que se puede reducir a la siguiente idea:

1. Genera tokens que podrían completar las secuencias.
2. Comprueba las puntuaciones de estos tokens con nuestro Modelo de Recompensa.
3. Actualiza nuestro modelo basado tanto en las puntuaciones como en las generaciones de nuestro modelo de *referencia* - que será nuestro modelo original antes de RLHF.

Es importante decir que esto se puede hacer tanto con LoRA como en QLoRA!!


In [22]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from peft import LoraConfig

rl_model_id = "HuggingFaceH4/zephyr-7b-alpha"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

base_model_rl = AutoModelForCausalLMWithValueHead.from_pretrained(
    rl_model_id,
    device_map={"": current_device},
    quantization_config=quant_config,
    peft_config=lora_config
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Necesitaremos configurar nuestro tokenizador y solucionar posibles problemas con `eos_token`.


In [23]:
rl_tokenizer = AutoTokenizer.from_pretrained(rl_model_id)

if getattr(rl_tokenizer, "pad_token", None) is None:
    rl_tokenizer.pad_token = rl_tokenizer.eos_token

### Conjunto de datos de entrenamiento

Para nuestro Modelo de Recompensa, hemos utilizado el conjunto de datos `hh-rlhf` de Anthropic - pero para nuestro entrenamiento PPO, usaremos el conjunto de datos [`allenai/real-toxicity-prompts`](https://huggingface.co/datasets/allenai/real-toxicity-prompts), que es simplemente una colección de prompts con salidas potencialmente tóxicas!

Como siempre, utilizaremos un subconjunto de estos para entrenar nuestro modelo hoy.


In [24]:
dataset_name="allenai/real-toxicity-prompts"

train_dataset = load_dataset(dataset_name, split="train")
train_dataset = train_dataset.select(range(1_000))

In [25]:
train_dataset

Dataset({
    features: ['filename', 'begin', 'end', 'challenging', 'prompt', 'continuation'],
    num_rows: 1000
})

### Formatting Prompts

Necesitamos preparar los prompts tal como:


```
Question: <<SAMPLE EXTRACTED FROM DATASET>>

Answer:
```

A continuación, filtraremos basándonos en secuencias largas y devolveremos nuestro conjunto de datos mapeado.



In [26]:
def build_dataset(
      tokenizer,
      dataset_name="allenai/real-toxicity-prompts",
  ):
    # Carga el conjunto de datos especificado con el nombre dataset_name, utilizando la partición "train".
    ds = load_dataset(dataset_name, split="train")
    original_columns = ds.column_names  # Guarda los nombres de las columnas originales del conjunto de datos.
    num_proc = 24  # Define el número de procesos para paralelizar el preprocesamiento.

    def preprocess_function(examples):
        new_examples = {
            "query": [],
            "input_ids": [],
        }
        # Procesa cada pregunta del conjunto de datos, añadiendo un prefijo y tokenizando.
        for question in examples["prompt"]:
            # Construye la consulta añadiendo "Question: " al principio y "\n\nAnswer: " al final.
            query = "Question: " + question["text"] + "\n\nAnswer: "
            # Tokeniza la consulta utilizando el tokenizador pasado a la función.
            tokenized_question = tokenizer(query, truncation=True)
            # Añade la consulta y sus identificadores de tokens procesados a new_examples.
            new_examples["query"].append(query)
            new_examples["input_ids"].append(tokenized_question["input_ids"])

        return new_examples

    # Aplica la función de preprocesamiento al conjunto de datos, eliminando las columnas originales.
    ds = ds.map(
        preprocess_function,
        batched=True,
        num_proc=num_proc,
        remove_columns=original_columns,
    )
    # Filtra las entradas para asegurarse de que la longitud de "input_ids" sea menor de 512.
    ds = ds.filter(lambda x: len(x["input_ids"]) < 512, batched=False)

    ds.set_format(type="torch")  # Cambia el formato del conjunto de datos para ser compatible con PyTorch.
    return ds  # Retorna el conjunto de datos preprocesado.

In [27]:
dataset = build_dataset(rl_tokenizer)

Map (num_proc=24):   0%|          | 0/99442 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Filter:   0%|          | 0/99442 [00:00<?, ? examples/s]

Este `collator` nos ayudará a llenar nuestra ventana de contexto de entrenamiento con tantos ejemplos como podamos ajustar!


In [28]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

### Configurando el PPOConfig

Veamos los hiper-parámetros:

- `steps` - cuántos pasos realizaremos en nuestro entrenamiento!
- `model_name` - bastante claro
- `learning_rate` - ¡con qué rapidez queremos aprender! Un valor pequeño `1.4e-5` debería estar bien aquí.
- `batch_size` - este valor podría ser tan grande como la capacidad de GPU que tengas!
- `ppo_epochs` - cuántas épocas queremos ejecutar PPO.
- `target_kl`, `init_kl_coef`, `adap_kl_ctrl` - estos son parámetros más avanzados de los que no nos preocuparemos hoy!


In [29]:
from trl import PPOConfig
config = PPOConfig(
    model_name=rl_model_id,
    learning_rate=1.4e-5,
    batch_size=32,
    mini_batch_size=1,
    gradient_accumulation_steps=4,
    optimize_cuda_cache=True,
    early_stopping=False,
    ppo_epochs=4,
    target_kl=0.1,
    init_kl_coef=0.2,
    adap_kl_ctrl=True
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


### Configurando el PPOTrainer

¡Todo lo que queda por hacer es configurar nuestro PPOTrainer!

Esto se hace de una manera muy similar a las otras clases `Trainer` de Hugging Face!


In [30]:
ppo_trainer = PPOTrainer(
    config,
    base_model_rl,
    ref_model=None,
    tokenizer=rl_tokenizer,
    dataset=dataset,
    data_collator=collator,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [31]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0

### Configuración del Modelo de Recompensa

Ahora que hemos entrenado nuestro Modelo de Recompensa, necesitamos poder utilizarlo durante el Entrenamiento PPO.

Utilizaremos los siguientes hiperparámetros para la consistencia.


In [32]:
sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16,
    "truncation": True,
}

Ahora podemos configurar un pipeline de sentimiento utilizando nuestro modelo de recompensa entrenado.


In [33]:
from transformers import pipeline

sentiment_pipe = pipeline(
    "sentiment-analysis",
    reward_model,
    device_map={"" : current_device},
    tokenizer=reward_model_tokenizer,
    return_token_type_ids=False,
)

### Configuración de Generación para el Modelo de Entrenamiento

Queremos asegurarnos de que nuestro modelo produzca una salida consistente cada vez, así que estableceremos nuestros `kwargs` de generación para asegurarnos de que así sea.


In [34]:
generation_kwargs = {
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": reward_model_tokenizer.pad_token_id,
    "eos_token_id": 100_000,
}

In [35]:
from trl.core import LengthSampler

output_min_length = 32
output_max_length = 128
output_length_sampler = LengthSampler(output_min_length, output_max_length)

Ahora, configuramos nuestro bucle de entrenamiento PPO.

Aquí tienen los pasos:

1. Genera tensores de respuesta de los modelos.
2. Decodifica las respuestas.
3. Calcula las Recompensas para las respuestas.
4. Actualiza nuestro modelo de entrenamiento.


10 min aprox en una A100

In [ ]:
from tqdm import tqdm

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if epoch >= config.total_ppo_epochs:
        break

    question_tensors = batch["input_ids"]

    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        length_sampler=output_length_sampler,
        **generation_kwargs,
    )

    batch["response"] = rl_tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[0]["score"]) for output in pipe_outputs]

    stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
10it [18:53, 109.12s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
25it [49:24, 113.46s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (35.64) exceeds threshold 10.00. Skipping batch.
  

In [ ]:
ppo_trainer.save_pretrained("rlhf_zephyr")

Cargamos el modelo. Recuerda que solo hemos entrenado los `adapters`, no todo el modelo.


In [ ]:
from peft import AutoPeftModelForCausalLM

rlhf_model = AutoPeftModelForCausalLM.from_pretrained(
    "rlhf_zephyr",
    device_map={"": current_device},
    quantization_config=quant_config,
)

Hacemos el merge entre los `adapters` y el modelo para poder hacer inferencia


In [ ]:
rlhf_merged_model = rlhf_model.merge_and_unload()

### Generando Salidas de Ejemplo

Ahora podemos crear una *pipeline* y ejecutar nuestro modelo base a través de 50 ejemplos de estos *prompts* potencialmente nocivos.


In [ ]:
import torch
from transformers import pipeline

rlhf_pipeline = pipeline("text-generation", model=rlhf_merged_model, tokenizer=rl_tokenizer)

In [ ]:
def generate_output_from_prompt(sample, pipe):
  messages = [
      {"role": "user", "content": sample["prompt"].strip()},
  ]
  prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
  outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
  return outputs[0]["generated_text"]

> Este script puede estar 10 minutos generando respuestas


In [ ]:
from tqdm import tqdm

rlhf_model_generations = []

for toxic_prompt in tqdm(toxic_prompt_list):
  rlhf_model_generations.append(generate_output_from_prompt(toxic_prompt, rlhf_pipeline))

In [ ]:
rlhf_model_generations[0]

In [ ]:
rlhf_model_generations_only_completions = []

for generation in rlhf_model_generations:
  rlhf_model_generations_only_completions.append(generation.split("<|assistant|>")[-1])

In [ ]:
rlhf_model_generations_only_completions[0]

Un cop hem recuperat les nostres respostes, podem utilitzar-les per determinar una puntuació global de "toxicitat".

Fixeu-vos que, en el fons, això està utilitzant un altre [LLM - facebook/roberta-hate-speech-dynabench-r4-target](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target)!

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -qU evaluate

In [ ]:
import evaluate

toxicity = evaluate.load("toxicity")

overall_results = toxicity.compute(predictions=rlhf_model_generations_only_completions)

In [ ]:
import numpy as np

np.mean(overall_results['toxicity'])

¡Wow! A pesar de realizar un entrenamiento bastante corto, ¡hemos logrado una reducción significativa de toxicidad!
